Agentic RAG
- RAG 를 수행하되, Agent 를 활용하여 RAG 를 수행한다면 이를 Agentic RAG 라고 부릅니다.

도구(Tools)
- Agent 가 활용할 도구를 정의하여 Agent 가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있습니다.
- Tavily Search 는 그 중 대표적인 검색 도구 입니다. 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성할 수 있습니다. 도구는 이처럼 검색 도구 뿐만아니라 Python 코드를 실행할 수 있는 도구, 직접 정의한 함수를 실행하는 도구 등 다양한 종류와 방법론을 제공합니다.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agents")


LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agents


In [3]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)


In [4]:
# 검색 결과를 가져옵니다.
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")


[{'url': 'https://www.youtube.com/watch?v=BpynjxkklPc&pp=ygUNI-q3gOyXveq1rOuniA%3D%3D',
  'content': '00:00 오프닝\n00:30 건물안\n01:25 춘식이 비밀박스\n01:45 춘식이 집\n03:18 죠르디 데스크\n04:04 튜브 휴식시간\n04:21 어피치 자동차\n04:59 크리스마스\n05:16 상품\n08:32 주간 베스트\n08:43 춘식도락\n09:28 골프\n10:08 무직타이거\n10:22 미피\n10:32 잔망 루피\n10:42 앙꼬\n10:58 총평\n\n\nKOREA TRAVEL\nnewly opened KAKAO FRIENDS Pangyo Agit in KOREA\n✔Store : Kakao Friends Pangyo Agit\n✔Address : 1F, 166, Pangyoyeok-ro, Bundang-gu, Seongnam-si, Gyeonggi-do, Republic of Korea\n✔Phone Number : 031-601-7225\n✔Opening Hours : Weekdays 10 ~ 21 / Weekends & holidays 12 ~ 20 [...] (ENG) 카카오프렌즈 판교아지트 전체둘러보기 춘식이 KAKAO FRIENDS KOREA TRAVEL \n 서울여자 Seoul Woman \n 13158 views \n 19 Nov 2022 \n #카카오프렌즈 #춘식이 #kakaofriends\n\n카카오프렌즈 판교아지트 전체둘러보기 !! 춘식이도 ~\n최근 신규오픈한 카카오본사 매장 🎉\nVIP 투어로 다녀왔습니다 🏃\u200d♀️\n\n✔지점명 : 카카오프렌즈 판교아지트\n✔주소 : 경기 성남시 분당구 판교역로 166 1층\n✔전화번호 : 031-601-7225\n✔영업시간 : 평일 10시 ~ 21시 / 주말공휴일 12시 ~ 20시 [...] ● 서울여자 Seoul Woman ●\n추가 컨텐츠는 인스타로 놀러오세요 👇👇👇\nhttp://w

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
ollama_embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    # model="chatfire/bge-m3:q8_0" # BGE-M3
)
vector = Chroma.from_documents(
    documents=split_docs, embedding=ollama_embeddings, collection_name="my_db"
)


# Retriever를 생성합니다.
retriever = vector.as_retriever()


/var/folders/5d/4yvbq7s16359fdr947szsx5h0000gn/T/ipykernel_7983/2927062215.py:16: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_embeddings = OllamaEmbeddings(


In [6]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke("삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘")

[Document(id='d21cbb47-c0fc-4e8c-a7db-e7007b753fb0', metadata={'author': 'dj', 'creationdate': '2023-12-08T13:28:38+09:00', 'creator': 'Hwp 2018 10.0.0.13462', 'moddate': '2023-12-08T13:28:38+09:00', 'page': 4, 'page_label': '5', 'pdfversion': '1.4', 'producer': 'Hancom PDF 1.3.0.542', 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23}, page_content='SPRi AI Brief |  \n2023-12 월호\n2G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의\nnG7이 첨단 AI 시스템을 개발하는 기업을 대상으로 AI 위험 식별과 완화를 위해 자발적인 \n채택을 권고하는 AI 국제 행동강령을 마련\nn행동강령은 AI 수명주기 전반에 걸친 위험 평가와 완화, 투명성과 책임성의 보장, 정보공유와 \n이해관계자 간 협력, 보안 통제, 콘텐츠 인증과 출처 확인 등의 조치를 요구KEY Contents\n£G7, 첨단 AI 시스템의 위험 관리를 위한 국제 행동강령 마련\nn주요 7개국(G7)* 은 2023 년 10월 30일 ‘히로시마 AI 프로세스 ’를 통해 AI 기업 대상의 AI 국제 \n행동강령 (International Code of Conduct for Advanced AI Systems) 에 합의\n∙G7은 2023 년 5월 일본 히로시마에서 개최된 정상회의에서 생성 AI에 관한 국제규범 마련과 \n정보공유를 위해 ‘히로시마 AI 프로세스 ’를 출범**\n∙기업의 자발적 채택을 위해 마련된 이번 행동강령은 기반모델과 생성 AI를 포함한 첨단 AI 시스템의 \n위험 식별과 완화에 필요한 조치를 포함\n* 주요 7개국(G7)은 미국, 일본, 

이제 우리가 검색을 수행할 인덱스를 채웠으므로, 이를 에이전트가 제대로 사용할 수 있는 도구로 쉽게 변환할 수 있습니다.

create_retriever_tool 함수로 retriever 를 도구로 변환합니다.

In [7]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)


Agent 가 사용할 도구 목록 정의
이제 두 가지를 모두 만들었으므로, Agent 가 사용할 도구 목록을 만들 수 있습니다.

tools 리스트는 search와 retriever_tool을 포함합니다.

In [8]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

Agent 생성
- 이제 도구를 정의했으니 에이전트를 생성할 수 있습니다.
- 먼저, Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의합니다.

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    api_key="ollama",
    model="llama3.2:1b",
    base_url="http://localhost:11434/v1",
)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


In [10]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

In [11]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False) # verbose=False 로 설정하여 중간 단계 출력을 생략하였습니다.

에이전트 실행하기

In [12]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()


In [13]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)


[도구 호출]
Tool: tavily_search_results_json
query: 2024年 프로 야구 플레이오프 진출 한 5 개 팀"
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024年 프로 야구 플레이오프 진출 한 5 개 팀"'}`



[관찰 내용]
Observation: [{'url': 'https://m.blog.naver.com/qhadldhaus98/223605904423', 'content': '야구 골수팬들은 당연히\n팀 순위가 머리 속에 있을 수 있지만\n라이트 팬들은 상위권이랑\n내가 응원하는 팀만 알지 어느팀이 몇위였더라?\n잘 모르실 수 있습니다.\n일단 2024 프로야구 순위를 먼저 적어보겠습니다.\n\u200b\n1.KIA 타이거즈\n2.삼성 라이온즈\n3.LG 트윈스\n4.두산 베어스\n5.KT 위즈\n6.SSG 랜더스\n7.롯데 자이언츠\n8.한화 이글스\n9.NC 다이노스\n10.키움 히어로즈\n\u200b\n올 해 프로야구 포스트시즌에 진출한 팀들은\nKIA 타이거즈 삼성 라이온즈 LG 트윈스\n두산 베어스 KT위즈입니다.\n\n출처 KBO\n\n출처 두산베어스\n\n출처 두산베어스\n\n출처 KT위즈\n\n출처 KT위즈\n\n2024 프로야구 일정 와일드카드 결정전 [...] 2024 프로야구 포스트시즌 일정의 그 첫 일정은\n4위 두산 베어스와 5위 KT 위즈와의 경기였습니다.\n첫 경기인 10월 2일 4:0 오늘 경기 1:0으로\n4위 두산 베어스를 꺾고 5위 KT 위즈가\nKBO 준플레이오프에 진출하게 되었습니다.\n5위가 4위를 이긴건 KBO 와일드카드 결정전 시작하고\n처음 있는 일이었습니다. KT위즈 새역사를 쓰다.\nKT위즈는 LG 트윈스와\n10월 5일 잠실에서 KBO준플레이오프\n1차전에서 만나게 됩니다.\n\n출처 LG 트윈스\n\n출처 LG 트윈스\n\u200b\n\n\n출처 KT위즈\n\n2024 KBO 준플레이오프 경기 일정 및 

In [14]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "삼성전자가 자체 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)


[도구 호출]
Tool: pdf_search
query: Samsung development of generative AI related information in the document
Log: 
Invoking: `pdf_search` with `{'query': 'Samsung development of generative AI related information in the document'}`



[관찰 내용]
Observation: 저해상도 이미지의 고해상도 전환도 지원
nIT 전문지 테크리퍼블릭 (TechRepublic) 은 온디바이스 AI가 주요 기술 트렌드로 부상했다며 , 
2024 년부터 가우스를 탑재한 삼성 스마트폰이 메타의 라마(Llama)2 를 탑재한 퀄컴 기기 및 구글 
어시스턴트를 적용한 구글 픽셀(Pixel) 과 경쟁할 것으로 예상
☞ 출처 : 삼성전자 , ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스 ’ 공개, 2023.11.08.
삼성전자 , ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.
TechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.

SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분

이전 대화내용 기억하는 Agent
- 이전의 대화내용을 기억하기 위해서는 RunnableWithMessageHistory 를 사용하여 AgentExecutor 를 감싸줍니다.

In [15]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)


In [16]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "삼성전자가 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)


[도구 호출]
Tool: pdf_search
query: Samsung_generationAI
Log: 
Invoking: `pdf_search` with `{'query': 'Samsung_generationAI'}`



[관찰 내용]
Observation: 1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육
알리바바 클라우드 , 최신 LLM ‘통이치엔원 2.0’ 공개
n알리바바 클라우드가 복잡한 지침 이해, 광고문구 작성, 추론, 암기 등에서 성능이 향상된 최신 
LLM ‘통이치엔원 2.0’을 공개
n알리바바 클라우드는 산업별로 특화된 생성 AI 모델을 공개하는 한편, 모델 개발과 애플리케이션 
구축 절차를 간소화하는 올인원 AI 모델 구축 플랫폼도 출시KEY Contents
£알리바바의 통이치엔원 2.0, 주요 벤치마크 테스트에서 여타 LLM 능가
n중국의 알리바바 클라우드가 2023 년 10월 31일 열린 연례 기술 컨퍼런스에서 최신 LLM ‘통이
치엔원 (Tongyi Qianwen) 2.0’을 공개
∙알리바바 클라우드는 통이치엔원 2.0이 2023 년 4월 출시된 1.0 버전보다 복잡한 지침 이해, 
광고문구 작성, 추론, 암기 등에서 성능이 향상되었다고 설명
∙통이치엔원 2.0은 언어 이해 테스트 (MMLU), 수학(GSM8k), 질문 답변(ARC-C) 과 같은 벤치마크 
테스트에서 라마(Llama-2-70B) 와 GPT-3.5 를 비롯한 주요 AI 모델을 능가 
∙통이치엔원 2.0은 알리바바 클라우드의 웹사이트와 모바일 앱을 통해 대중에 제공되며 개발자는 
API를 통해 사용 가능 
n알리바바 클라우드는 여러 산업 영역에서 생성 AI를 활용해 사업 성과를 개선할 수 있도록 지원
하는 산업별 모델도 출시
∙산업 영역은 고객지원 , 법률 상담, 의료, 금융, 문서관리 , 오디오와 동영상 관리, 코드 개발, 캐릭터 
제작을 포함
n알리바바 클라우드는 급증하는 생성 AI 수요에 대응해 모델 개발과 애플리케이션 구축 절차를 
간소화하는 올

In [17]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)


[도구 호출]
Tool: tavily_search_results_json
query: 삼성전자가 developing creation AI related information
Log: 
Invoking: `tavily_search_results_json` with `{'query': '삼성전자가 developing creation AI related information'}`



[관찰 내용]
Observation: [{'url': 'https://research.samsung.com/artificial-intelligence', 'content': 'At Samsung Research, our goal is to develop new AI that enriches the customer experience for the billions of users and devices under the Samsung brand. This entails creating AI that is not only aware of how our devices are used but can also anticipate and meet user needs proactively. [...] To achieve this, we are developing a generative AI model named Samsung Gauss. Inspired by Carl Friedrich Gauss, a legendary mathematician known for establishing the normal distribution theory crucial to machine learning and AI, Samsung Gauss symbolizes our vision to leverage global phenomena and knowledge. This approach aims to harness AI’s power to significantly improve consumer lives worldwid

Agent 템플릿

In [18]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의합니다 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = Chroma.from_documents(
    documents=split_docs, embedding=ollama_embeddings, collection_name="my_db"
)

# Retriever를 생성합니다.
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

########## 2. LLM 을 정의합니다 ##########
# LLM 모델을 생성합니다.
llm = ChatOpenAI(
    api_key="ollama",
    model="llama3.2:1b",
    base_url="http://localhost:11434/v1",
)

########## 3. Prompt 를 정의합니다 ##########

# Prompt 를 정의합니다 - 이 부분을 수정할 수 있습니다!
# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의합니다 ##########

# 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()


In [19]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)




> Entering new None chain...
[도구 호출]
Tool: pdf_search
query: PDF search for functions
Log: 
Invoking: `pdf_search` with `{'query': 'PDF search for functions'}`




Invoking: `pdf_search` with `{'query': 'PDF search for functions'}`


18영국 옥스퍼드 인터넷 연구소 , AI 기술자의 임금이 평균 21% 높아
n옥스퍼드 인터넷 연구소의 연구에 따르면 특정 기술의 경제적 가치는 다른 기술과 결합 
가능성이 높을수록 높게 평가됨 
nAI의 확산은 기술의 경제적 가치에 크게 영향을 미치며 , AI 기술을 가진 근로자는 평균 21%, 
최대 40% 높은 임금을 받을 수 있음  KEY Contents
£AI 기술 중 머신러닝 , 텐서플로우 , 딥러닝의 임금 프리미엄이 높게 평가
n옥스퍼드 인터넷 연구소 (Oxford Internet Institute) 가 2023년 10월 24일 962개 기술과 2만 5천 
명을 대상으로 한 연구에서 AI를 포함한 주요 기술의 경제적 가치를 분석한 결과를 발표 
∙연구에 따르면 한 기술의 경제적 가치는 근로자의 여타 역량과 얼마나 잘 결합하는지를 보여주는 
‘상보성 (complementarity)’ 에 따라 결정됨
∙특정 기술은 다른 기술과 결합 가능성이 높을수록 경제적 가치가 높아지며 , 일례로 데이터 분석과 같은 
기술은 여타 고부가가치 기술과 결합할 수 있어 가치가 높지만 , 사진 리터칭 같은 기술은 특정 기술과만 
결합할 수 있어 가치가 낮게 평가됨 
∙대부분 직업은 여러 기술의 조합이 필요하며 , 근로자의 재교육에서 경제적 효율성을 높이려면 기존 기술과 
신기술 간 상보성을 극대화할 필요
nAI의 확산은 기술의 경제적 가치에 크게 영향을 미치는 요소로 , AI 기술을 가진 근로자는 평균적으로  
21% 높은 임금을 획득 가능
∙AI 기술 중 근

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)


In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변에 한국 시리즈 일정을 추가하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)
